In [1]:
import pvdeg

## Pysam

rundown on pysam...

https://nrel-pysam.readthedocs.io/en/main/inputs-from-sam.html

In [ ]:
weather_db = 'PSM3'
weather_id = (25.783388, -80.189029)
weather_arg = {
    'api_key': 'DEMO_KEY',
    'email': 'user@mail.com',
    'names': 'tmy',
    'attributes': [],
    'map_variables': True
    }

weather_nsrdb, meta_nsrdb = pvdeg.weather.get(weather_db, weather_id, **weather_arg)

In [2]:
weather_pvigs, meta_pvgis = pvdeg.weather.get(database="PVGIS", id=(25.783388, -80.189029))

In [3]:
weather_pvigs

,temp_air,relative_humidity,ghi,dni,dhi,IR(h),wind_speed,wind_direction,pressure
time(UTC),,,,,,,,,
2006-01-01 00:00:00+00:00,23.26,47.01,0.0,0.00,0.0,22.89,3.54,169.0,101639.0
2006-01-01 01:00:00+00:00,23.07,54.77,0.0,0.00,0.0,24.45,3.51,173.0,101669.0
2006-01-01 02:00:00+00:00,22.87,62.53,0.0,0.00,0.0,26.02,3.47,174.0,101679.0
2006-01-01 03:00:00+00:00,22.68,70.30,0.0,0.00,0.0,27.58,3.43,173.0,101709.0
2006-01-01 04:00:00+00:00,22.49,78.06,0.0,0.00,0.0,29.14,3.39,170.0,101689.0
...,...,...,...,...,...,...,...,...,...
2008-12-31 19:00:00+00:00,24.24,8.19,468.0,386.74,241.0,15.07,3.74,318.0,101490.0
2008-12-31 20:00:00+00:00,24.04,15.95,344.0,318.58,193.0,16.63,3.70,322.0,101410.0
2008-12-31 21:00:00+00:00,23.85,23.72,288.0,684.34,71.0,18.20,3.66,326.0,101430.0


In [5]:
from pvdeg import utilities
weather_df = weather_pvigs
meta = meta_pvgis
model = "pysamv1"
model_default = 'FlatPlatePVCommercial'
# ARGS ================================

import PySAM.Pvwattsv8 as pv8
import PySAM.Pvsamv1 as pv1
import PySAM.Utilityrate5 as UtilityRate
import PySAM.Grid as Grid 
import PySAM.Cashloan as Cashloan
import json

weather_df = utilities.add_time_columns_tmy(weather_df=weather_df)

# load defaults from jsons


# https://nrel-pysam.readthedocs.io/en/main/modules/Pvwattsv8.html
# https://nrel-pysam.readthedocs.io/en/main/modules/Pvsamv1.html
if model == "pvwatts8": 
    pysam_model = pv8.default(model_default) # PVWattsCommerical
elif model == "pysamv1":
    pysam_model = pv1.default(model_default) # FlatPlatePVCommercial

if True: # add check args
    grid = Grid.from_existing(pysam_model)
    ur = UtilityRate.from_existing(pysam_model)
    cl = Cashloan.from_existing(pysam_model)


solar_resource = {
    'lat': meta['latitude'],
    'lon': meta['longitude'],
    'tz': meta['tz'] if 'tz' in meta.keys() else 0,
    'elev': meta['altitude'],
    'year': list(weather_df['Year']),
    'month': list(weather_df['Month']),
    'day': list(weather_df['Day']),
    'hour': list(weather_df['Hour']),
    'minute': list(weather_df['Minute']),
    'dn': list(weather_df['dni']),
    'df': list(weather_df['dhi']),
    'wspd': list(weather_df['wind_speed']),
    'tdry': list(weather_df['temp_air']),
    'alb' : weather_df['albedo'] if 'albedo' in weather_df.columns.values else [0.2]*len(weather_df) # this could be done more wisely
}

pysam_model.SolarResource.solar_resource_data = solar_resource
pysam_model.execute()
outputs = pysam_model.Outputs.export()

In [ ]:
outputs['annual_energy']